# Clustering Online Retail Data with Gaussian Mixture Model

### Import libraries

In [2]:
using GaussianMixtures
using XLSX
using DataFrames
using Dates
using Statistics
using Clustering
using PyPlot

### Import data

In [3]:
xf = XLSX.readxlsx("Online Retail.xlsx")
sh = xf["Online Retail"] # Read sheet
data = sh[:]
df = DataFrame(XLSX.readtable("Online Retail.xlsx", "Online Retail")...) # Convert to DataFrame

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate
,Any,Any,Any,Any,Any
1,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01T08:26:00
2,536365,71053,WHITE METAL LANTERN,6,2010-12-01T08:26:00
3,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01T08:26:00
4,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01T08:26:00
5,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01T08:26:00
6,536365,22752,SET 7 BABUSHKA NESTING BOXES,2,2010-12-01T08:26:00
7,536365,21730,GLASS STAR FROSTED T-LIGHT HOLDER,6,2010-12-01T08:26:00
8,536366,22633,HAND WARMER UNION JACK,6,2010-12-01T08:28:00
9,536366,22632,HAND WARMER RED POLKA DOT,6,2010-12-01T08:28:00


In [4]:
size(df)

(541909, 8)

In [5]:
describe(df)

,variable,mean,min,median,max,nmissing
,Symbol,Union…,Any,Union…,Any,Int64
1,InvoiceNo,,,,,0
2,StockCode,,,,,0
3,Description,,,,,1454
4,Quantity,9.55225,-80995,3.0,80995,0
5,InvoiceDate,,2010-12-01T08:26:00,,2011-12-09T12:50:00,0
6,UnitPrice,4.61111,-11062.1,2.08,38970.0,0
7,CustomerID,15287.7,12346,15152.0,18287,135080
8,Country,,Australia,,Unspecified,0


In [50]:
# Remove invalid rows
remove_missing_df = dropmissing!(df);

In [7]:
# Unique CustomerIDs
size(unique(df.CustomerID))

(4372,)

In [8]:
describe(remove_missing_df)

,variable,mean,min,median,max
,Symbol,Union…,Any,Union…,Any
1,InvoiceNo,,,,
2,StockCode,,,,
3,Description,,4 PURPLE FLOCK DINNER CANDLES,,ZINC WIRE SWEETHEART LETTER TRAY
4,Quantity,12.0613,-80995,5.0,80995
5,InvoiceDate,,2010-12-01T08:26:00,,2011-12-09T12:50:00
6,UnitPrice,3.46047,0.0,1.95,38970.0
7,CustomerID,15287.7,12346,15152.0,18287
8,Country,,Australia,,Unspecified


### Preprocess data

In [52]:
# Convert date column to DateTime
remove_missing_df.date = map((x) -> DateTime(x), remove_missing_df.InvoiceDate);

We will group the data by CustomerID. The three features we will use for the clusters are recency, frequency, and monetary. Recency is the number of days since the most recent transaction in the dataset. The frequency is the number of transactions for that user. Monetary is the total amount of money that the customer has spent. These features allow for an understanding of the customer's spending habits at the store.

In [78]:
gdf = groupby(remove_missing_df, :CustomerID);
max_date = DateTime(maximum(remove_missing_df.date));
df4 = combine(gdf, :date => maximum => :max_date);
df4.recency = map((x) -> (Dates.value(max_date - DateTime(x)) / (1000 * 60 * 60 * 24)), df4.max_date);
df4.frequency = combine(gdf, nrow).nrow;
df4.monetary = combine(gdf, 
    AsTable([:Quantity, :UnitPrice]) =>
    x -> (sum(x.Quantity) * sum(x.UnitPrice))).Quantity_UnitPrice_function;

In [84]:
# Standardize the data
df4.recency = df4.recency .- mean(df4.recency) ./ std(df4.recency);
df4.frequency = df4.frequency .- mean(df4.frequency) ./ std(df4.frequency);
df4.monetary = df4.monetary .- mean(df4.monetary) ./ std(df4.monetary);
df4

,CustomerID,max_date,recency,frequency,monetary
,Any,DateTime,Float64,Float64,Float64
1,17850,2011-02-10T14:38:00,301.016,311.6,2.0731e6
2,13047,2011-11-08T12:10:00,30.1191,195.6,1.0817e6
3,12583,2011-12-07T08:07:00,1.28785,250.6,3.96352e6
4,13748,2011-09-05T09:45:00,94.2198,27.5997,49124.0
5,15100,2011-01-13T17:09:00,328.911,5.59972,3810.55
6,15291,2011-11-14T11:02:00,24.1663,108.6,1.14313e6
7,14688,2011-12-02T12:26:00,6.10799,358.6,2.42681e6
8,17809,2011-11-23T13:00:00,15.0844,63.5997,1.80784e6
9,15311,2011-12-09T12:00:00,-0.873953,2490.6,2.3642e8


### Gaussian mixture model clustering

We will determine the best number of clusters to describe the data using the elbow method. Perform Gaussian mixture model clustering with increasing number of clusters. The best number of clusters will be the elbow of the average log likelihood plot.

In [87]:
# Collect the 
lls = Float64[]
for n in 1:6
    gm = GMM(n, Array(df4[:, 3:5]), kind = :full);
    append!(lls, avll(gm, Array(df4[:, 3:5])));
end

K-means converged with 2 iterations (objv = 1.7422287752292518e18)


┌ Info: Initializing GMM, 2 Gaussians diag covariance 3 dimensions using 4372 data points
└ @ GaussianMixtures /Users/pascalewalters/.julia/packages/GaussianMixtures/1pQcF/src/train.jl:79
┌ Info: K-means with 2000 data points using 2 iterations
│ 250.0 data points per parameter
└ @ GaussianMixtures /Users/pascalewalters/.julia/packages/GaussianMixtures/1pQcF/src/train.jl:140
┌ Warning: Too low occupancy count 1.0 for Gausian 2
└ @ GaussianMixtures /Users/pascalewalters/.julia/packages/GaussianMixtures/1pQcF/src/train.jl:265
┌ Warning: Too low occupancy count 1.0 for Gausian 2
└ @ GaussianMixtures /Users/pascalewalters/.julia/packages/GaussianMixtures/1pQcF/src/train.jl:265
┌ Warning: Too low occupancy count 1.0 for Gausian 2
└ @ GaussianMixtures /Users/pascalewalters/.julia/packages/GaussianMixtures/1pQcF/src/train.jl:265
┌ Warning: Too low occupancy count 1.0 for Gausian 2
└ @ GaussianMixtures /Users/pascalewalters/.julia/packages/GaussianMixtures/1pQcF/src/train.jl:265
┌ Warning: Too

K-means converged with 2 iterations (objv = 5.5028543974595e17)


┌ Warning: Too low occupancy count 1.0 for Gausian 2
└ @ GaussianMixtures /Users/pascalewalters/.julia/packages/GaussianMixtures/1pQcF/src/train.jl:265
┌ Info: Initializing GMM, 3 Gaussians diag covariance 3 dimensions using 4372 data points
└ @ GaussianMixtures /Users/pascalewalters/.julia/packages/GaussianMixtures/1pQcF/src/train.jl:79
┌ Info: K-means with 3000 data points using 2 iterations
│ 250.0 data points per parameter
└ @ GaussianMixtures /Users/pascalewalters/.julia/packages/GaussianMixtures/1pQcF/src/train.jl:140
┌ Warning: Too low occupancy count 1.0 for Gausian 2
└ @ GaussianMixtures /Users/pascalewalters/.julia/packages/GaussianMixtures/1pQcF/src/train.jl:265
┌ Warning: Too low occupancy count 1.0 for Gausian 2
└ @ GaussianMixtures /Users/pascalewalters/.julia/packages/GaussianMixtures/1pQcF/src/train.jl:265
┌ Warning: Too low occupancy count 1.0 for Gausian 2
└ @ GaussianMixtures /Users/pascalewalters/.julia/packages/GaussianMixtures/1pQcF/src/train.jl:265
┌ Warning: Too

K-means converged with 3 iterations (objv = 3.791530144579525e17)


┌ Warning: Too low occupancy count 1.0 for Gausian 2
└ @ GaussianMixtures /Users/pascalewalters/.julia/packages/GaussianMixtures/1pQcF/src/train.jl:265
┌ Warning: Too low occupancy count 1.0 for Gausian 2
└ @ GaussianMixtures /Users/pascalewalters/.julia/packages/GaussianMixtures/1pQcF/src/train.jl:265
┌ Warning: Too low occupancy count 1.0 for Gausian 2
└ @ GaussianMixtures /Users/pascalewalters/.julia/packages/GaussianMixtures/1pQcF/src/train.jl:265
┌ Info: Initializing GMM, 4 Gaussians diag covariance 3 dimensions using 4372 data points
└ @ GaussianMixtures /Users/pascalewalters/.julia/packages/GaussianMixtures/1pQcF/src/train.jl:79
┌ Info: K-means with 4000 data points using 3 iterations
│ 250.0 data points per parameter
└ @ GaussianMixtures /Users/pascalewalters/.julia/packages/GaussianMixtures/1pQcF/src/train.jl:140
┌ Warning: Too low occupancy count 1.0 for Gausian 2
└ @ GaussianMixtures /Users/pascalewalters/.julia/packages/GaussianMixtures/1pQcF/src/train.jl:265
┌ Warning: Too

K-means converged with 3 iterations (objv = 1.3359668793157654e17)


┌ Warning: Too low occupancy count 1.0 for Gausian 3
└ @ GaussianMixtures /Users/pascalewalters/.julia/packages/GaussianMixtures/1pQcF/src/train.jl:265
┌ Warning: Too low occupancy count 1.0 for Gausian 2
└ @ GaussianMixtures /Users/pascalewalters/.julia/packages/GaussianMixtures/1pQcF/src/train.jl:265
┌ Warning: Too low occupancy count 1.0 for Gausian 3
└ @ GaussianMixtures /Users/pascalewalters/.julia/packages/GaussianMixtures/1pQcF/src/train.jl:265
┌ Warning: Too low occupancy count 1.0 for Gausian 2
└ @ GaussianMixtures /Users/pascalewalters/.julia/packages/GaussianMixtures/1pQcF/src/train.jl:265
┌ Warning: Too low occupancy count 1.0 for Gausian 3
└ @ GaussianMixtures /Users/pascalewalters/.julia/packages/GaussianMixtures/1pQcF/src/train.jl:265
┌ Warning: Too low occupancy count 1.0 for Gausian 2
└ @ GaussianMixtures /Users/pascalewalters/.julia/packages/GaussianMixtures/1pQcF/src/train.jl:265
┌ Warning: Too low occupancy count 1.0 for Gausian 3
└ @ GaussianMixtures /Users/pascale

LoadError: PosDefException: matrix is not positive definite; Cholesky factorization failed.

In [88]:
lls

4-element Vector{Float64}:
 -10.557428757345358
 -10.36826601214841
  -8.52218286235137
  -8.502906982890863

Using the elbow method, the best number of clusters is 4.

In [93]:
# Perform GMM clustering with 4 clusters
gm = GMM(4, Array(df4[:, 3:5]), kind = :full); 

K-means converged with 2 iterations (objv = 1.619628872528443e17)


┌ Info: Initializing GMM, 4 Gaussians diag covariance 3 dimensions using 4372 data points
└ @ GaussianMixtures /Users/pascalewalters/.julia/packages/GaussianMixtures/1pQcF/src/train.jl:79
┌ Info: K-means with 4000 data points using 2 iterations
│ 250.0 data points per parameter
└ @ GaussianMixtures /Users/pascalewalters/.julia/packages/GaussianMixtures/1pQcF/src/train.jl:140
┌ Warning: Too low occupancy count 0.0 for Gausian 2
└ @ GaussianMixtures /Users/pascalewalters/.julia/packages/GaussianMixtures/1pQcF/src/train.jl:265
┌ Warning: 3 pathological elements normalized
└ @ GaussianMixtures /Users/pascalewalters/.julia/packages/GaussianMixtures/1pQcF/src/train.jl:69
┌ Warning: Too low occupancy count 0.0 for Gausian 2
└ @ GaussianMixtures /Users/pascalewalters/.julia/packages/GaussianMixtures/1pQcF/src/train.jl:265
┌ Warning: 3 pathological elements normalized
└ @ GaussianMixtures /Users/pascalewalters/.julia/packages/GaussianMixtures/1pQcF/src/train.jl:69
┌ Warning: Too low occupancy c

In [94]:
# Assign samples to classes
prob_pos = gmmposterior(gm, Array(df4[:, 3:5]))[1];
ass = [argmax(prob_pos[i, :]) for i = 1:size(df4, 1)];

In [95]:
# Get the means of the GMM clusters
means(gm)

4×3 Matrix{Float64}:
 109.856      37.9433  61852.1
   0.0         0.0         0.0
   7.11648  1076.38        1.26408e8
  26.839     218.995       1.53808e6

In [96]:
# Find how many data points are in each of the clusters
ass = [argmax(prob_pos[i, :]) for i=1:size(df4, 1)];
println("Class 1: ", sum(a == 1 for a in ass))
println("Class 2: ", sum(a == 2 for a in ass))
println("Class 3: ", sum(a == 3 for a in ass))
println("Class 4: ", sum(a == 4 for a in ass))

Class 1: 3313
Class 2: 0
Class 3: 80
Class 4: 979


The four clusters represent four customer segments that have been set based on their frequency, recency, and monetary values. Four clusters were selected based on the elbow method. Looking at the cluster means, Class 3 has the highest recency, frequency, and monetary values. Class 1 has the lowest frequency, recency, and monetary values. Class 4 has intermediate values. Class 2 does not contain any data samples.